In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [19]:
from pybloomfilter import BloomFilter
import tqdm
import urllib
fname = '../../data/kb/dbpedia_t2ksubset/all_surfaceforms.txt'
bloom = '../../data/kb/dbpedia_t2ksubset/all_surfaceforms.5pct.bloom'
size = 16710167
bf = BloomFilter(size, 0.05, bloom)
with open(fname) as f:
    for li, word in enumerate(tqdm.tqdm(f, total=size)):
        if '%' in word:
            word = urllib.parse.unquote(word).lower()
        word = word.rstrip()
        bf.add(word)

print(word, word in bf)
bf.close()

bf = BloomFilter.open(bloom)
print(word, word in bf)

100%|██████████| 16710167/16710167 [01:26<00:00, 193066.48it/s]


zeuxis True
zeuxis True


In [1]:
from dask.distributed import Client
Client(address = 'tcp://192.168.62.207:8686')
# Client(dashboard_address = ':8687')
# Client()

/export/scratch1/home/kruit/venv2/lib/python3.7/site-packages/distributed/client.py:1130: VersionMismatchWarning: Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| dask    | 2.30.0        | 2.30.0        | 2.20.0        |
| python  | 3.7.4.final.0 | 3.7.4.final.0 | 3.7.3.final.0 |
+---------+---------------+---------------+---------------+
Notes: 
-  python: Variation is sometimes ok, sometimes not. It depends on your workloads
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Client Scheduler: tcp://192.168.62.207:8686 Dashboard: http://192.168.62.207:8687/status,Cluster Workers: 6 Cores: 192 Memory: 1.62 TB


In [2]:
%%time
import dask.bag as db
db.read_text('/bigstore/kruit/webtables/englishCorpus_2015-07_filtered/*.jsonl').count().compute()

CPU times: user 2min 41s, sys: 13.9 s, total: 2min 54s
Wall time: 9min 30s


49658538

In [2]:
%%time
import dask.bag as db
import json, itertools
tables = db.read_text('/bigstore/kruit/webtables/englishCorpus_2015-07_filtered/xa*.jsonl').map(json.loads)

sample_n = 1000
sample = tables.map_partitions(lambda xs: list(itertools.islice(xs, sample_n)))
sample = sample.foldby(lambda x: hash(str(x['relation'])), binop = lambda tot, x:x).map(lambda x: x[1])
sample = sample.repartition(tables.npartitions)

import urllib
from collections import Counter

key = lambda x: (urllib.parse.urlparse(x['url']).netloc, next(zip(*x['relation']), None))
counts = dict(sample.foldby(key, initial=0, binop = lambda tot,x: tot+1).compute())
dict(Counter(counts).most_common(10))

CPU times: user 313 ms, sys: 78.6 ms, total: 392 ms
Wall time: 6.61 s


{('www.cappex.com', ('Categories', 'Comments')): 56,
 ('www.reef.org', ('', 'Expert', 'Novice', 'Total')): 24,
 ('www.google.com',
  ('Citing Patent',
   'Filing date',
   'Publication date',
   'Applicant',
   'Title')): 24,
 ('www.bigstockphoto.com', ('', 'px | in | cm', 'Credits')): 23,
 ('www.google.com', ('Date', 'Code', 'Event', 'Description')): 19,
 ('www.dpreview.com', ('Post (hide subjects)', 'Posted by', 'When', '')): 18,
 ('eprints.ucm.es', ('Directive', 'Local Value', 'Master Value')): 18,
 ('www.usms.org', ('#', 'Name', 'Age', 'Club', 'LMSC', 'Time')): 17,
 ('www.appliancefactoryparts.com',
  ('Ref.', 'Image', 'Part No & Description', 'Price', '')): 17,
 ('www.timeanddate.com', ('Location', 'Local time')): 17}

In [3]:
sample.count().compute()

23085

In [5]:
from takco.extract import from_wdc
from takco.util import preview
from takco import TableSet, DaskHashBag, HashBag

# ts = HashBag(sample.compute()).pipe(from_wdc)
ts = DaskHashBag(sample).pipe(from_wdc)
ts = TableSet.cluster(ts, addcontext=['pgTitle'])
ts = TableSet.reshape(ts, restructure=False, compound_splitter_config={'class':'SuffixCompoundSplitter'})
ts = sorted(ts, key=lambda x: -x['numDataRows'])
print(f"Got {len(ts)} tables")
preview(ts, ntables=25)

Got 7007 tables
